# Preprocessing

In [22]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion SNR

In [33]:
import numpy as np
from scipy.signal import welch

def calculate_snr_fft(audio_signal, sample_rate=16000, noise_duration=0.5):
    """
    Schätzt das Signal-Rausch-Verhältnis (SNR) basierend auf FFT und spektraler Analyse.

    Quelle:
        Harris, F., & Dick, C. (2012). 
        SNR Estimation Techniques for Low SNR Signals. 
        In *The 15th International Symposium on Wireless Personal Multimedia Communications* 
        (pp. 24-27). IEEE Xplore. 
        DOI: 10.1109/WPMC.2012.6370441

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).
        noise_duration (float): Länge der Rauschschätzphase in Sekunden (Standard: 0.5s).

    Returns:
        float: Signal-to-Noise Ratio (SNR) in dB.
    """
    # Anzahl der Samples für die Noise-Region
    noise_end_sample = min(int(noise_duration * sample_rate), len(audio_signal))
    noise_segment = audio_signal[:noise_end_sample]

    # Berechne PSD mit Welch's Methode
    f_signal, psd_signal = welch(audio_signal, fs=sample_rate, nperseg=1024)
    f_noise, psd_noise = welch(noise_segment, fs=sample_rate, nperseg=1024)

    # Mittlere Leistung des Signals und des Rauschens
    signal_power = np.mean(psd_signal)
    noise_power = np.mean(psd_noise)

    # Verhindere Division durch Null
    if noise_power == 0:
        return 100.0  # Bedeutet praktisch kein Rauschen

    # Berechnung des SNR in dB
    snr_value = 10 * np.log10(signal_power / noise_power)

    return snr_value

# Test

In [34]:
import os

# SNR für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)

        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)

        # SNR berechnen
        snr_value = calculate_snr_fft(audio_signal)

        print(f"SNR ({file_name}): {snr_value:.2f} dB")

SNR (_noise_pink.wav): -0.03 dB
SNR (_noise_white.wav): -0.08 dB
SNR (_signal_constant.wav): 100.00 dB
SNR (_signal_silence.wav): 100.00 dB
SNR (_signal_sine.wav): 0.00 dB
SNR (_snr_03.wav): 8.70 dB
SNR (_snr_10.wav): 14.18 dB
SNR (_snr_20.wav): 22.13 dB
SNR (example1.wav): 25.93 dB
SNR (example2.wav): 26.04 dB
SNR (example3.wav): 26.23 dB
